In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np
import pandas as pd
import re

In [1]:
import pickle
with open('clean_line_df.pkl', 'rb') as f:
    line_df = pickle.load(f)

In [2]:
line_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 485604 entries, 1 to 15651
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   author             485604 non-null  category
 1   title              485604 non-null  object  
 2   line               485604 non-null  object  
 3   words              485604 non-null  object  
 4   length_in_words    485604 non-null  int64   
 5   lexical_diversity  485604 non-null  float64 
dtypes: category(1), float64(1), int64(1), object(3)
memory usage: 23.2+ MB


In [9]:
corpus_lines = line_df.line.to_list()

In [10]:
corpus_lines[0:10]

['philosophic',
 'in its complex ovoid emptiness',
 'a skillful pundit coined it as a sort',
 'of stopgap doorstop for those',
 'quaint equations',
 'romans never',
 'dreamt of in form completely clever',
 'and discrete—a mirror come unsilvered',
 'loose watch face without the works',
 'a hollowed globe']

In [11]:
print(len(corpus_lines))
print(corpus_lines[0])
corpus_string = ' '.join(corpus_lines)

485604
philosophic


In [14]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(corpus_lines)
# input_sequences = tokenizer.texts_to_sequences(corpus_lines)
total_words = len(tokenizer.word_index) + 1
print(total_words)
input_sequences = []
for line in corpus_lines:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
        

121325


In [15]:
max_sequence_length = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_length, padding='pre'))

In [17]:
xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = tf.keras.utils.to_categorical(labels)

In [19]:
model = Sequential()
model.add(Embedding(total_words, 80, input_length=max_sequence_length-1))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50))
model.add(tf.keras.layers.Dropout(0.1))
model.add(Dense(total_words/20))
model.add(Dense(total_words, activation='softmax'))
model.summary()

TypeError: only size-1 arrays can be converted to Python scalars